# CUHK [STAT3009](https://www.bendai.org/STAT3009/) Notebook7: Neural Collaborative Filtering

### Install ``TensorFlow`` and ``Keras``
- TensorFlow: https://www.tensorflow.org/install
- Keras: Keras comes packaged with TensorFlow 2 as tensorflow.keras (https://keras.io/about/)
- If you use Apple M1: https://naturale0.github.io/2021/01/29/setting-up-m1-mac-for-both-tensorflow-and-pytorch

Credit: The notebook is adapted from https://calvinfeng.gitbook.io/machine-learning-notebook/supervised-learning/recommender/neural_collaborative_filtering

## Introduction to Deep learning with Keras
	- Model: input -> layers -> output
	- Loss: find an appropriate loss function for your problem
	- Algo: SGD, Adam, ...
	- Data: Define the model, then feed the data
	- metric: final evaluation or something you care

## Example 1: [Imbalanced classification: credit card fraud detection](https://keras.io/examples/structured_data/imbalanced_classification/)

- Author: fchollet
- Date created: 2019/05/28
- Last modified: 2020/04/17
- Description: Demonstration of how to handle highly imbalanced classification problems.

In [1]:
# https://keras.io/examples/structured_data/imbalanced_classification/

import csv
import numpy as np

# Get the real data from https://www.kaggle.com/mlg-ulb/creditcardfraud/
fname = "/home/ben/dataset/creditcard.csv"

all_features = []
all_targets = []
with open(fname) as f:
    for i, line in enumerate(f):
        if i == 0:
            print("HEADER:", line.strip())
            continue  # Skip header
        fields = line.strip().split(",")
        all_features.append([float(v.replace('"', "")) for v in fields[:-1]])
        all_targets.append([int(fields[-1].replace('"', ""))])
        if i == 1:
            print("EXAMPLE FEATURES:", all_features[-1])

features = np.array(all_features, dtype="float32")
targets = np.array(all_targets, dtype="uint8")
print("features.shape:", features.shape)
print("targets.shape:", targets.shape)

HEADER: "Time","V1","V2","V3","V4","V5","V6","V7","V8","V9","V10","V11","V12","V13","V14","V15","V16","V17","V18","V19","V20","V21","V22","V23","V24","V25","V26","V27","V28","Amount","Class"
EXAMPLE FEATURES: [0.0, -1.3598071336738, -0.0727811733098497, 2.53634673796914, 1.37815522427443, -0.338320769942518, 0.462387777762292, 0.239598554061257, 0.0986979012610507, 0.363786969611213, 0.0907941719789316, -0.551599533260813, -0.617800855762348, -0.991389847235408, -0.311169353699879, 1.46817697209427, -0.470400525259478, 0.207971241929242, 0.0257905801985591, 0.403992960255733, 0.251412098239705, -0.018306777944153, 0.277837575558899, -0.110473910188767, 0.0669280749146731, 0.128539358273528, -0.189114843888824, 0.133558376740387, -0.0210530534538215, 149.62]
features.shape: (284807, 30)
targets.shape: (284807, 1)


In [2]:
num_val_samples = int(len(features) * 0.2)
train_features = features[:-num_val_samples]
train_targets = targets[:-num_val_samples]
val_features = features[-num_val_samples:]
val_targets = targets[-num_val_samples:]

print("Number of training samples:", len(train_features))
print("Number of validation samples:", len(val_features))

Number of training samples: 227846
Number of validation samples: 56961


In [3]:
mean = np.mean(train_features, axis=0)
train_features -= mean
val_features -= mean
std = np.std(train_features, axis=0)
train_features /= std
val_features /= std

In [4]:
## Build binary classifcation model
from tensorflow import keras

model = keras.Sequential(
    [
        keras.layers.Dense(
            256, activation="relu", input_shape=(train_features.shape[-1],)
        ),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(256, activation="relu"),
        keras.layers.Dropout(0.3),
        keras.layers.Dense(1, activation="sigmoid"),
    ]
)
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 256)               7936      
_________________________________________________________________
dense_1 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout (Dropout)            (None, 256)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 256)               65792     
_________________________________________________________________
dropout_1 (Dropout)          (None, 256)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 257       
Total params: 139,777
Trainable params: 139,777
Non-trainable params: 0
__________________________________________________

In [5]:
metrics = [
    keras.metrics.BinaryAccuracy(name='acc'),
    keras.metrics.AUC(name='auc')
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-2), loss="binary_crossentropy", metrics=metrics
)

callbacks = [keras.callbacks.EarlyStopping( 
    monitor='val_auc', min_delta=0, patience=5, verbose=1, 
    mode='auto', baseline=None, restore_best_weights=True)]

model.fit(
    train_features,
    train_targets,
    batch_size=2048,
    epochs=30,
    verbose=2,
    callbacks=callbacks,
    validation_data=(val_features, val_targets),
)

Epoch 1/30
112/112 - 2s - loss: 0.0152 - acc: 0.9960 - auc: 0.8984 - val_loss: 0.0031 - val_acc: 0.9995 - val_auc: 0.9262
Epoch 2/30
112/112 - 1s - loss: 0.0036 - acc: 0.9992 - auc: 0.9592 - val_loss: 0.0036 - val_acc: 0.9996 - val_auc: 0.8998
Epoch 3/30
112/112 - 1s - loss: 0.0031 - acc: 0.9993 - auc: 0.9571 - val_loss: 0.0033 - val_acc: 0.9996 - val_auc: 0.9130
Epoch 4/30
112/112 - 2s - loss: 0.0029 - acc: 0.9993 - auc: 0.9668 - val_loss: 0.0035 - val_acc: 0.9996 - val_auc: 0.8997
Epoch 5/30
112/112 - 1s - loss: 0.0028 - acc: 0.9993 - auc: 0.9704 - val_loss: 0.0034 - val_acc: 0.9996 - val_auc: 0.8865
Epoch 6/30
112/112 - 1s - loss: 0.0026 - acc: 0.9993 - auc: 0.9730 - val_loss: 0.0026 - val_acc: 0.9996 - val_auc: 0.9329
Epoch 7/30
112/112 - 1s - loss: 0.0025 - acc: 0.9994 - auc: 0.9718 - val_loss: 0.0044 - val_acc: 0.9996 - val_auc: 0.8998
Epoch 8/30
112/112 - 1s - loss: 0.0026 - acc: 0.9994 - auc: 0.9730 - val_loss: 0.0028 - val_acc: 0.9996 - val_auc: 0.9128
Epoch 9/30
112/112 - 1s 

In [6]:
## Back to array
pred_prob = model.predict(val_features)
pred_label = 1*(pred_prob > .5)

## Example 2: [Collaborative Filtering for Movie Recommendations](https://keras.io/examples/structured_data/collaborative_filtering_movielens/)

- Author: Siddhartha Banerjee
- Date created: 2020/05/24
- Last modified: 2020/05/24
- Description: Recommending movies using a model trained on Movielens dataset.

## Pre-process the ML-100K raw data
- check the `user_id` and `item_id`: mapping `item_id` to a continuous sequence based on `sklean.preprocessing`
- use `sklearn.model_selection.train_test_split` to generate `train` and `test` dataset

In [7]:
import numpy as np
import pandas as pd

df = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
del df['timestamp']
## mapping 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['movieId'] = le.fit_transform(df['movieId'])
df['userId'] = le.fit_transform(df['userId'])
## generate train / test dataset
from sklearn.model_selection import train_test_split
dtrain, dtest = train_test_split(df, test_size=0.33, random_state=42)
## save real ratings for test set for evaluation.
test_rating = np.array(dtest['rating'])
## remove the ratings in the test set to simulate prediction
dtest = dtest.drop(columns='rating')


In [8]:
# tran_pair, train_rating
train_pair = dtrain[['userId', 'movieId']].values
train_rating = dtrain['rating'].values
# test_pair
test_pair = dtest[['userId', 'movieId']].values
n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1

### Create the model
We embed both users and movies in to 50-dimensional vectors.

The model computes a match score between user and movie embeddings via a dot product, and adds a per-movie and per-user bias.

In [9]:
import pandas as pd
import numpy as np
from zipfile import ZipFile
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from pathlib import Path
import matplotlib.pyplot as plt

In [10]:
class LFactorNet(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(LFactorNet, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.user_bias = layers.Embedding(num_users, 1)
        self.glb_bias = tf.Variable(0., trainable=True) 
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.movie_bias = layers.Embedding(num_movies, 1)

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        user_bias = self.user_bias(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        movie_bias = self.movie_bias(inputs[:, 1])
        dot_user_movie = tf.tensordot(user_vector, movie_vector, 2)
        # Add all the components (including bias)
        x = dot_user_movie + user_bias + movie_bias + self.glb_bias
        return x

In [11]:
model = LFactorNet(num_users=n_user, num_movies=n_item, embedding_size=50)


metrics = [
    keras.metrics.MeanAbsoluteError(name='mae'),
    keras.metrics.RootMeanSquaredError(name='rmse')
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=metrics
)

callbacks = [keras.callbacks.EarlyStopping( 
    monitor='val_rmse', min_delta=0, patience=5, verbose=1, 
    mode='auto', baseline=None, restore_best_weights=True)]

history = model.fit(
    x=train_pair,
    y=train_rating,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=.2,
)

Epoch 1/50
845/845 [==============================] - 3s 3ms/step - loss: 6.1625 - mae: 1.8174 - rmse: 2.2690 - val_loss: 2.2280 - val_mae: 0.8652 - val_rmse: 1.0799
Epoch 2/50
845/845 [==============================] - 3s 4ms/step - loss: 1.9292 - mae: 0.8411 - rmse: 1.0469 - val_loss: 1.8998 - val_mae: 0.8542 - val_rmse: 1.0609
Epoch 3/50
845/845 [==============================] - 3s 4ms/step - loss: 1.7552 - mae: 0.8094 - rmse: 1.0124 - val_loss: 1.7486 - val_mae: 0.8173 - val_rmse: 1.0202
Epoch 4/50
845/845 [==============================] - 3s 3ms/step - loss: 1.6177 - mae: 0.7777 - rmse: 0.9757 - val_loss: 1.6147 - val_mae: 0.7753 - val_rmse: 0.9823
Epoch 5/50
845/845 [==============================] - 3s 3ms/step - loss: 1.4673 - mae: 0.7475 - rmse: 0.9413 - val_loss: 1.4885 - val_mae: 0.7733 - val_rmse: 0.9716
Epoch 6/50
845/845 [==============================] - 3s 3ms/step - loss: 1.3111 - mae: 0.7206 - rmse: 0.9118 - val_loss: 1.3167 - val_mae: 0.7412 - val_rmse: 0.9408
Epoc

In [12]:
## make prediction
pred_rating = model.predict(test_pair).flatten()
print(pred_rating)
print('rmse: LFactorNet: %.3f' %np.sqrt(np.mean((pred_rating - test_rating)**2)))

[2.749611  3.44922   2.1344523 ... 4.125555  3.2430491 4.173697 ]
rmse: LFactorNet: 0.892


## Example 3: Neural Collaborative Filtering in MovieLens dataset

Credit: The notebook is adapted from https://calvinfeng.gitbook.io/machine-learning-notebook/supervised-learning/recommender/neural_collaborative_filtering

## Pre-process the ML-100K raw data
- check the `user_id` and `item_id`: mapping `item_id` to a continuous sequence based on `sklean.preprocessing`
- use `sklearn.model_selection.train_test_split` to generate `train` and `test` dataset

In [13]:
import numpy as np
import pandas as pd

df = pd.read_csv('./dataset/ml-latest-small/ratings.csv')
del df['timestamp']
## mapping 
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
df['movieId'] = le.fit_transform(df['movieId'])
df['userId'] = le.fit_transform(df['userId'])
## generate train / test dataset
from sklearn.model_selection import train_test_split
dtrain, dtest = train_test_split(df, test_size=0.33, random_state=42)
## save real ratings for test set for evaluation.
test_rating = np.array(dtest['rating'])
## remove the ratings in the test set to simulate prediction
dtest = dtest.drop(columns='rating')

In [ ]:
# tran_pair, train_rating
train_pair = dtrain[['userId', 'movieId']].values
train_rating = dtrain['rating'].values
# test_pair
test_pair = dtest[['userId', 'movieId']].values
n_user, n_item = max(train_pair[:,0].max(), test_pair[:,0].max())+1, max(train_pair[:,1].max(), test_pair[:,1].max())+1

### Create NCF Model

In [20]:
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.layers import Embedding, Flatten, Input, Dropout, Dense, Concatenate
from tensorflow.keras.optimizers import Adam
from IPython.display import SVG
from tensorflow import keras
from tensorflow.keras import layers

In [45]:
class NCF(keras.Model):
    def __init__(self, num_users, num_movies, embedding_size, **kwargs):
        super(NCF, self).__init__(**kwargs)
        self.num_users = num_users
        self.num_movies = num_movies
        self.embedding_size = embedding_size
        self.user_embedding = layers.Embedding(
            num_users,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.movie_embedding = layers.Embedding(
            num_movies,
            embedding_size,
            embeddings_initializer="he_normal",
            embeddings_regularizer=keras.regularizers.l2(1e-2),
        )
        self.concatenate = layers.Concatenate()
        self.dense1 = layers.Dense(100, name='fc-1', activation='relu')
        self.dense2 = layers.Dense(50, name='fc-2', activation='relu')
        self.dense3 = layers.Dense(1, name='fc-3', activation='relu')

    def call(self, inputs):
        user_vector = self.user_embedding(inputs[:, 0])
        movie_vector = self.movie_embedding(inputs[:, 1])
        concatted_vec = self.concatenate([user_vector, movie_vector])
        fc_1 = self.dense1(concatted_vec)
        fc_2 = self.dense2(fc_1)
        fc_3 = self.dense3(fc_2)
        return fc_3

## Select `loss function`, `metrics`, `algorithm`

In [46]:
model = NCF(num_users=n_user, num_movies=n_item, embedding_size=50)

metrics = [
    keras.metrics.MeanAbsoluteError(name='mae'),
    keras.metrics.RootMeanSquaredError(name='rmse')
]

model.compile(
    optimizer=keras.optimizers.Adam(1e-3), 
    loss=tf.keras.losses.MeanSquaredError(), 
    metrics=metrics
)

# from tensorflow.keras.utils import plot_model
# plot_model(model, to_file='model.png')

In [47]:
callbacks = [keras.callbacks.EarlyStopping( 
    monitor='val_rmse', min_delta=0, patience=5, verbose=1, 
    mode='auto', baseline=None, restore_best_weights=True)]

history = model.fit(
    x=train_pair,
    y=train_rating,
    batch_size=64,
    epochs=50,
    verbose=1,
    validation_split=.2,
)

Epoch 1/50
845/845 [==============================] - 4s 4ms/step - loss: 1.5728 - mae: 0.8344 - rmse: 1.1406 - val_loss: 0.9819 - val_mae: 0.7092 - val_rmse: 0.9188
Epoch 2/50
845/845 [==============================] - 3s 4ms/step - loss: 0.9226 - mae: 0.6944 - rmse: 0.8945 - val_loss: 0.9458 - val_mae: 0.7053 - val_rmse: 0.9121
Epoch 3/50
845/845 [==============================] - 3s 3ms/step - loss: 0.8897 - mae: 0.6843 - rmse: 0.8827 - val_loss: 0.9183 - val_mae: 0.6914 - val_rmse: 0.9046
Epoch 4/50
845/845 [==============================] - 3s 3ms/step - loss: 0.8638 - mae: 0.6737 - rmse: 0.8697 - val_loss: 0.9091 - val_mae: 0.6895 - val_rmse: 0.8987
Epoch 5/50
845/845 [==============================] - 3s 4ms/step - loss: 0.8588 - mae: 0.6692 - rmse: 0.8629 - val_loss: 0.9031 - val_mae: 0.6912 - val_rmse: 0.8931
Epoch 6/50
845/845 [==============================] - 3s 4ms/step - loss: 0.8476 - mae: 0.6616 - rmse: 0.8542 - val_loss: 0.9075 - val_mae: 0.6873 - val_rmse: 0.8906
Epoc

In [48]:
## make prediction
pred_rating = model.predict(test_pair).flatten()
print(pred_rating)
print('rmse: LFactorNet: %.3f' %np.sqrt(np.mean((pred_rating - test_rating)**2)))

[2.9420114 3.3594353 2.0358317 ... 3.8942363 3.1720738 3.9475594]
rmse: LFactorNet: 0.882
